In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/24 15:31:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/24 15:31:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [29]:
FILE = '../../de_bootcamp/week5/data/raw/fhvhv/2021/02/fhvhv_tripdata_2021-02.csv'
TARGET_DIR= '../../de_bootcamp/week5/data/parquet/'

In [9]:
df = spark.read \
    .option("header", "true") \
    .csv(FILE)

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [12]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [17]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [11]:
# df.write.parquet('zones')

In [10]:
import pandas as pd

In [11]:
df_fhv_pd = pd.read_csv(FILE, nrows=1000)

In [13]:
df_fhv_pd

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02764,2021-02-01 00:10:40,2021-02-01 00:21:09,35,39,NaN
1,HV0003,B02764,2021-02-01 00:27:23,2021-02-01 00:44:01,39,35,NaN
2,HV0005,B02510,2021-02-01 00:28:38,2021-02-01 00:38:27,39,91,NaN
3,HV0005,B02510,2021-02-01 00:43:37,2021-02-01 01:23:20,91,228,NaN
4,HV0003,B02872,2021-02-01 00:08:42,2021-02-01 00:17:57,126,250,NaN
...,...,...,...,...,...,...,...
995,HV0003,B02872,2021-02-01 00:35:49,2021-02-01 00:40:59,79,144,NaN
996,HV0005,B02510,2021-02-01 00:16:35,2021-02-01 00:41:17,254,208,NaN
997,HV0005,B02510,2021-02-01 00:42:49,2021-02-01 00:57:57,208,240,NaN
998,HV0003,B02682,2021-02-01 00:23:08,2021-02-01 00:41:46,165,188,NaN


In [16]:
df_fhv_pd.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [18]:
spark.createDataFrame(df_fhv_pd).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [19]:
from pyspark.sql import types

In [20]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [21]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv(FILE)

In [22]:
df = df.repartition(24)

In [30]:
TARGET_FOLDER=f'{TARGET_DIR}fhvhv/2021/02/'

In [31]:
df = df.write.parquet(TARGET_FOLDER)

In [34]:
df = spark.read.parquet(f'{TARGET_DIR}fhvhv/2021/02/')

In [35]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [36]:
from pyspark.sql import functions as F

In [37]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [47]:
df.select('hvfhs_license_num','pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').filter(df.hvfhs_license_num== 'HV0003').count()

8290758

In [58]:
from pyspark.sql.functions import year, month, dayofmonth

In [66]:
df.select( dayofmonth('pickup_datetime').alias('day')).filter(df.day == 15).show()

+---+
|day|
+---+
|  6|
|  5|
|  4|
|  3|
|  4|
|  4|
|  5|
|  3|
|  3|
|  2|
|  5|
|  4|
|  2|
|  4|
|  5|
|  4|
|  4|
|  5|
|  5|
|  4|
+---+
only showing top 20 rows



In [67]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [70]:
df.groupBy(dayofmonth('pickup_datetime').alias('day')).count().orderBy('day', ascending = False).show()

+---+------+
|day| count|
+---+------+
| 28|422116|
| 27|509383|
| 26|500049|
| 25|436556|
| 24|412556|
| 23|392696|
| 22|382811|
| 21|399763|
| 20|497072|
| 19|459887|
| 18|372878|
| 17|415856|
| 16|362596|
| 15|367170|
| 14|460661|
| 13|509331|
| 12|469162|
| 11|428288|
| 10|403476|
|  9|398476|
+---+------+
only showing top 20 rows



In [102]:
from pyspark.sql.functions import to_timestamp, col
df2 = df .withColumn('DiffInSeconds', col( 'dropoff_datetime').cast("long")- col('pickup_datetime').cast("long"))

In [103]:
df2.orderBy('DiffInSeconds', ascending = False).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|DiffInSeconds|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|         247|          41|   null|        75540|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|         242|         254|   null|        57221|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|         188|          55|   null|        44039|
|           HV0003|              B02864|2021-02-03 20:24:25|2021-02-04 07:41:58|          51|         147|   null|        40653|
|           HV0003|              B02887|2021-02-19 23:17:44|2021-02-20 09:44:01|         210|    

In [105]:
df2.groupBy('dispatching_base_num').count().orderBy('count', ascending = False).show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [ ]:
df2.groupBy('dispatching_base_num').count().orderBy('count', ascending = False).show()